In [8]:
import paddle.fluid as fluid
import paddle
import numpy as np
paddle.enable_static()

使用静态图模式

In [9]:
x=fluid.data(name='x',shape=[None,1],dtype='float32')
hidden=fluid.layers.fc(input=x,size=100,act='relu')
hidden=fluid.layers.fc(input=hidden,size=100,act='relu')
net=fluid.layers.fc(input=hidden,size=1,act=None)
infer_program=fluid.default_main_program().clone(for_test=True)

建模，建立预测函数，这里的infer_program将预测模型进行了封装，paddle的program机制就是写到哪里存到哪里

In [10]:
y=fluid.data(name='y',shape=[None,1],dtype='float32')
cost=fluid.layers.square_error_cost(input=net,label=y)
avg_cost=fluid.layers.mean(cost)
optimizer=fluid.optimizer.SGDOptimizer(learning_rate=0.01)
opts=optimizer.minimize(avg_cost)

编写损失函数，并用优化算法进行优化，这里也属于建模

In [11]:
place=fluid.CUDAPlace(0)
exe=fluid.Executor(place)
exe.run(fluid.default_startup_program())

[]

设置工作区，建立执行器，并初始化执行器，这里只是建执行器，并没有封装program，这里的初始化程序是环境默认的

In [12]:
x_data=np.array([[1.0],[2.0],[3.0],[4.0],[5.0]]).astype('float32')
y_data=np.array([[3.0],[5.0],[7.0],[9.0],[11.0]]).astype('float32')

编写喂给神经网络的数据

In [13]:
for pass_id in range(100):
    train_cost=exe.run(program=fluid.default_main_program(),feed={x.name:x_data, y.name:y_data},
    fetch_list=[avg_cost])
    print("Pass:%d, Cost:%0.5f" %(pass_id,train_cost[0]))

开始训练，这里100个循环表示训练100次，fetch_list决定了返回值，列表内的变量必须是定义在模型中的变量，所使用的main_program
是环境的默认程序，默认将本程序所有的代码封装

In [14]:
test_data=np.array([[6.0],[7.0]]).astype('float32')
result=exe.run(program=infer_program,
feed={x.name:test_data},
fetch_list=[net])
print(result[0][0][0])
print(result[0][1][0])

12.884837
14.875077


用之前封装的infer_program进行预测，这里result得说明一下：第一维度0是为在列表中找到net，找到net以后由于net的数据shape是（2，1）由test_data决定，
所以后面的[0][0]或者[1][0]由自己的需要决定，这里绝对不可能是[0][1]

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 